# Supply Stacks

## Part 1

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
Finally, one crate is moved from stack 1 to stack 2:

        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('day_05_input.txt', 'r') as f:
    data = f.readlines()

In [3]:
data[:15]

['    [C]         [Q]         [V]    \n',
 '    [D]         [D] [S]     [M] [Z]\n',
 '    [G]     [P] [W] [M]     [C] [G]\n',
 '    [F]     [Z] [C] [D] [P] [S] [W]\n',
 '[P] [L]     [C] [V] [W] [W] [H] [L]\n',
 '[G] [B] [V] [R] [L] [N] [G] [P] [F]\n',
 '[R] [T] [S] [S] [S] [T] [D] [L] [P]\n',
 '[N] [J] [M] [L] [P] [C] [H] [Z] [R]\n',
 ' 1   2   3   4   5   6   7   8   9 \n',
 '\n',
 'move 2 from 4 to 6\n',
 'move 4 from 5 to 3\n',
 'move 6 from 6 to 1\n',
 'move 4 from 1 to 4\n',
 'move 4 from 9 to 4\n']

In [17]:
stacks = []
directions = []

#split stacks and directions
for dat in data:
    dat = dat.strip('\n') 
    
    if dat.startswith('move'):
        directions.append(dat)
    else:
        if dat != '':
            stacks.append(dat)

In [18]:
#remove row number from stacks
stacks = stacks[:-1] 
stacks

['    [C]         [Q]         [V]    ',
 '    [D]         [D] [S]     [M] [Z]',
 '    [G]     [P] [W] [M]     [C] [G]',
 '    [F]     [Z] [C] [D] [P] [S] [W]',
 '[P] [L]     [C] [V] [W] [W] [H] [L]',
 '[G] [B] [V] [R] [L] [N] [G] [P] [F]',
 '[R] [T] [S] [S] [S] [T] [D] [L] [P]',
 '[N] [J] [M] [L] [P] [C] [H] [Z] [R]']

In [31]:
stack_start = []

#reverse the stacks
for stack in stacks[::-1]:
    stack_one = []

    #save box letter for each stack
    for i, s in enumerate(stack):
        if i in (1,5,9,13,17,21,25,29,33):
            stack_one.append(s)
    
    #make a list of lists
    stack_start.append(stack_one)

stack_start = pd.DataFrame(stack_start)
stack_start.columns = range(1,10) #renumber index to match stacks
stack_start = stack_start.sum().str.strip() #squish columns together
stack_start

1        NRGP
2    JTBLFGDC
3         MSV
4      LSRCZP
5    PSLVCWDQ
6     CTNWDMS
7       HDGWP
8    ZLPHSCMV
9     RPFLWGZ
dtype: object

> the top of each stack is on the right most box

In [38]:
#format the directions by pulling out the numbers only
directions_clean = pd.Series(directions)
directions_clean = directions_clean.str.split(' ', expand = True)
directions_clean = directions_clean[[1,3,5]]
directions_clean.columns = ['num', 'start', 'end']
directions_clean

,num,start,end
0,2,4,6
1,4,5,3
2,6,6,1
3,4,1,4
4,4,9,4
...,...,...,...
497,7,1,5
498,1,3,1
499,2,8,3
500,1,5,7


In [39]:
stack_end = stack_start.copy()

#iterate through each row to apply the crate directions
for i, row in directions_clean.iterrows():
    num = int(row['num'])
    start = int(row['start']) 
    end = int(row['end'])
    
    #grab letters to move
    to_move = stack_end[start][-num:]
    #remove letters
    stack_end[start] = stack_end[start][:-num]
    #add letters (reverse them first)
    stack_end[end] = stack_end[end] + to_move[::-1] 
    
stack_end

1    LSTCSRZZPFDSWPNDPMGRRVV
2                       PGHP
3                        WPC
4                  LDLBTCHQD
5                    CWFLVNM
6                        MSS
7                          L
8                         GW
9                       GCZJ
dtype: object

In [40]:
#get the right most letters, as they are the top of the stacks
stack_end.str[-1].sum()

'VPCDMSLWJ'

## Part 2

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
Moving a single crate from stack 2 to stack 1 behaves the same as before:

[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:

        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:

        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:

        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?

In [41]:
stack_end = stack_start.copy()

#iterate through each row to apply the crate directions
for i, row in directions_clean.iterrows():
    num = int(row['num'])
    start = int(row['start']) 
    end = int(row['end'])
    
    #grab letters to move
    to_move = stack_end[start][-num:]
    #remove letters
    stack_end[start] = stack_end[start][:-num]
    #add letters
    stack_end[end] = stack_end[end] + to_move
    
stack_end

1    FWPDPHLGJDCSGNVTSMQVSCT
2                       RPLP
3                        HPW
4                  ZPLLSMBVC
5                    SLZFZRG
6                        WRN
7                          C
8                         DC
9                       WMDG
dtype: object

In [43]:
#get the right most letters, as they are the top of the stacks
stack_end.str[-1].sum()

'TPWCGNCCG'